In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
## Load all the questions and labels

with open("generated_questions_labels.txt", "r") as f:
    lines = f.readlines()

questions = list()
labels = list()
levels = list()

for line in lines:
    line = line.split("\n")[0]
    line = line.split(" ||| ")
    questions.append(line[0].split("QUESTION: ")[1].strip())
    labels.append(line[1].split("LABEL: ")[1].strip())

    # Levels of strength of evidence were additionally generated but were not used in further experiments.
    levels.append(line[2].split("LEVEL: ")[1].strip())
    


In [ ]:
import openai
import time 

## Prepare the messages for answering the questions by each LLM.
## Most LLMs were called through Together AI API or hosted locally and queried.

all_messages = list()
for idx in range(len(questions)):

    system_message = "You're a helpful AI assistant helping answer clinical and medical questions based on your best knowledge."
    user_message_begin = f'''
           Please answer this clinical question only with SUPPORTED (if the question is supported by the clinical research) or REFUTED (if the hypothesis is refuted by the current clinical research) or NOT ENOUGH INFORMATION (if there is insufficient evidence for the question in current research). Please give your output in form of LABEL: (label) . Briefly explain your answer.

    QUESTION:
    '''

    finalstring = ""
    finalstring += user_message_begin
    finalstring += questions[idx]  + "\n"

    messages =  [  
            {'role':'system', 
             'content': system_message},    
            {'role':'user', 
             'content': f"{finalstring}"},  
    ] 
    
    all_messages.append(messages)


In [ ]:
client = openai.OpenAI(
  api_key="xyz",
  base_url="https://api.together.xyz/v1",
)

def query_llm(prompt):
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo", # set model version
        max_tokens = 512,
        messages=prompt, # provide prompt in chat format
        temperature=0) # set model temperature = 0
    return response


start_time = time.time()
with open("llama33-70b_answers.txt", "w") as f:
    curr_idx = 0

    for msgs in all_messages:
        response = query_llm(msgs)
        result = response.choices[0].message.content.strip()
        result = result.replace("\n", " ||| ")
        f.write(result)
        f.write("\n")
        curr_idx += 1

        # Track time.
        if curr_idx % 1000 == 0:
            end_time = time.time()
            print(curr_idx, end_time-start_time)
            start_time = time.time()